In [4]:
from fractions import Fraction
from functools import lru_cache
from collections import defaultdict
import pprint

def N(m):
    return m * (m - 3) // 2

def C_m(m, a):
    b = m + 2 - a
    return m if a != b else m // 2

def S(a, b):
    return int(a == 3) + int(b == 3)

@lru_cache(None)
def dist(polygons, player):
    # Total diagonals available across all polygons
    N_total = sum(N(m) for m in polygons)
    if N_total == 0:
        return {0: Fraction(1, 1)}
    
    result = defaultdict(Fraction)
    # Iterate over each polygon
    for idx, m in enumerate(polygons):
        Nm = N(m)
        if Nm == 0:
            continue
        
        # Only consider splits a ≤ b to avoid double-counting
        a_max = (m + 2) // 2
        for a in range(3, a_max + 1):
            b = m + 2 - a
            count = C_m(m, a)
            instant = S(a, b)
            # Probability of choosing one of these diagonals among all
            prob_choice = Fraction(count, N_total)
            
            # Build new multiset of polygons after drawing this diagonal
            new_polys = list(polygons)
            new_polys.pop(idx)
            new_polys.extend([a, b])
            new_polys.sort()
            
            # Recurse for the opponent's turn
            sub_dist = dist(tuple(new_polys), 2 if player == 1 else 1)
            
            # Combine distributions
            for delta_sub, p_sub in sub_dist.items():
                if player == 1:
                    delta = instant + delta_sub
                else:
                    delta = -instant + delta_sub
                result[delta] += prob_choice * p_sub
    
    return dict(result)

distribution_8 = dist((8,), 1)
win_probability = sum(p for d, p in distribution_8.items() if d > 0)

print("Distribution for Delta_8 (gap : probability):")
pprint.pprint(distribution_8)
print("\nProbability Player 1 wins:", win_probability, "≈", float(win_probability))


Distribution for Delta_8 (gap : probability):
{-2: Fraction(4, 165),
 0: Fraction(478, 1155),
 2: Fraction(53, 105),
 4: Fraction(2, 35)}

Probability Player 1 wins: 59/105 ≈ 0.5619047619047619
